In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
findspark.init()

In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Yield Prediction")\
        .config('spark.ui.port','4050')\
        .getOrCreate()
spark

In [7]:
!git clone https://github.com/ronanmccormack-ca/ProgrammingforBigData.git

Cloning into 'ProgrammingforBigData'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 41 (delta 8), reused 36 (delta 3), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [11]:
df = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/inventory_data.csv',header='true',inferSchema=True)

In [12]:
df.show()

+------------------+----------------+-------------------------------+--------+--------------------+-----------+--------------+-----------+--------------------+-------------+--------------------+------------------+--------------------+---------------------+---------------------+-----------------+-------------------+-------------------+------------------+----+----+----+----+----+----+----+
|measure_event_type|measure_event_id|measure_event_performed_on_date|batch_id|          batch_name|cultivar_id| cultivar_name|location_id|       location_name|work_order_id|     work_order_name|work_order_type_id|work_order_type_name|work_order_start_date|work_order_close_date|substance_type_id|substance_type_name|measure_event_value|measure_event_unit|_c19|_c20|_c21|_c22|_c23|_c24|_c25|
+------------------+----------------+-------------------------------+--------+--------------------+-----------+--------------+-----------+--------------------+-------------+--------------------+------------------+-----

In [13]:
df.describe().show()

+-------+------------------+------------------+-------------------------------+------------------+--------------------+------------------+-------------+-----------------+--------------------+-----------------+--------------------+------------------+--------------------+---------------------+---------------------+------------------+--------------------+-------------------+------------------+----+----+----+----+----+----+----+
|summary|measure_event_type|  measure_event_id|measure_event_performed_on_date|          batch_id|          batch_name|       cultivar_id|cultivar_name|      location_id|       location_name|    work_order_id|     work_order_name|work_order_type_id|work_order_type_name|work_order_start_date|work_order_close_date| substance_type_id| substance_type_name|measure_event_value|measure_event_unit|_c19|_c20|_c21|_c22|_c23|_c24|_c25|
+-------+------------------+------------------+-------------------------------+------------------+--------------------+------------------+----

In [26]:
co2_sensor_1 = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/co2_sensor_1.csv',header='true',inferSchema=True)

In [33]:
co2_sensor_1.describe().show()

+-------+--------------------+--------+------------------+-------------------+------------------+--------------------+-----------------+---------------+--------------------+---------+------------------+-------------------+
|summary|     DataMessageGUID|SensorID|       Sensor Name|               Date|             Value|     Formatted Value|          Battery|       Raw Data|        Sensor State|GatewayID|   Signal Strength|            Voltage|
+-------+--------------------+--------+------------------+-------------------+------------------+--------------------+-----------------+---------------+--------------------+---------+------------------+-------------------+
|  count|               49910|   49910|             49910|              49910|             49910|               49910|            49910|          49910|               49910|    49910|             49910|              49910|
|   mean|                null|595683.0|              null|               null|1131.3500901622922|           

In [35]:
co2_sensor_1.show()

+--------------------+--------+------------------+-------------------+-----+--------------------+-------+-------------+------------+---------+----------+---------------+-------+
|     DataMessageGUID|SensorID|       Sensor Name|               Date|Value|     Formatted Value|Battery|     Raw Data|Sensor State|GatewayID|Alert Sent|Signal Strength|Voltage|
+--------------------+--------+------------------+-------------------+-----+--------------------+-------+-------------+------------+---------+----------+---------------+-------+
|54aff7bb-d97c-4ec...|  595683|CO2 Meter - 595683|2021-04-29 13:26:00|  512|CO2 Instantaneous...|     73|512|541|0|359|           0|   942254|     false|             71|   2.64|
|e2c27bad-fc36-4ca...|  595683|CO2 Meter - 595683|2021-04-29 13:16:00|  522|CO2 Instantaneous...|     73|522|541|0|383|           0|   942254|     false|             67|   2.64|
|d186c2b5-0453-40e...|  595683|CO2 Meter - 595683|2021-04-29 13:06:00|  520|CO2 Instantaneous...|     73|520|5

AnalysisException: ignored